<a href="https://colab.research.google.com/github/EMPEROR82/PClubSecyRecruitment-Task-5/blob/main/FlightAllocation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
uploaded = files.upload()


Saving passengers.csv to passengers.csv
Saving flights.csv to flights.csv
Saving canceled.csv to canceled.csv


In [ ]:
from google.colab import files
uploaded = files.upload()


Saving passengers.csv to passengers (1).csv
Saving flights.csv to flights (1).csv
Saving canceled.csv to canceled (1).csv


In [ ]:
import pandas as pd
import networkx as nx
import time
import csv
from google.colab import files


start = time.time()
# Load data from the CSV files
passengers = pd.read_csv('/content/passengers.csv')
flights = pd.read_csv('/content/flights.csv')
canceled_flights = pd.read_csv('/content/canceled.csv')

reallocated_passengers = [] #list of allocated passengers with their route details . This data will be returned in allocate.csv


#Required data for stats.csv
affected_passengers = int(0)
alloted_passengers = int(0)
total_layovers = int(0)
total_time_diff = int(0)


# Create a list of flights
flight_list = []

# Define the Flight class
class Flight:
    def __init__(self, flight_id, departure_airport, arrival_airport, departure_time, arrival_time, capacity):
        self.flight_id = flight_id
        self.departure_airport = departure_airport
        self.arrival_airport = arrival_airport
        self.departure_time = departure_time
        self.arrival_time = arrival_time
        self.capacity = int(capacity)
        self.passengers = []  # List to store passengers
        self.empty_space = self.capacity  # Keeps track of the empty space in a flight
        self.canceled = False
        self.possible_routes = []
        self.allocated_flights = []

    def add_passenger(self, passenger):
        if len(self.passengers) < self.capacity:
            self.passengers.append(passenger)
            self.empty_space -= 1
            return True
        else:
            return False

    def cancel_flight(self):
        self.canceled = True

    def allocate_flights(self) :
        #this function greedily allocate the flights to the passengers and then updates the capacities of flights which have been used up in the allocation
        global total_layovers
        global total_time_diff
        global alloted_passengers
        inf = int(100000)

        passenger_count = int(len(self.passengers))
        z = int(0) #iterator in passengers
        k = int(len(self.possible_routes))
        y = int(0) #iterator in possible_routes

        def compare(x) :
          l = int(len(x[0]))
          last_flight = int(x[1][-1])
          time_diff = abs(int(self.arrival_time) - int(flight_list[last_flight].arrival_time))
          return ( l , time_diff )

        self.possible_routes = sorted( self.possible_routes , key = compare )
        #print(self.possible_routes)
        while z<passenger_count and y<k :
          route_flow = inf
          for intermediate_flight in self.possible_routes[y][1] :
            if route_flow > int(flight_list[int(intermediate_flight)].empty_space) :
              route_flow = int(flight_list[int(intermediate_flight)].empty_space)

          if passenger_count-z < route_flow :
            route_flow = passenger_count-z

          for i in range (0,route_flow) :
            reallocated_passengers.append([self.passengers[i+z] , len(self.possible_routes[y][1]) , self.possible_routes[y][1]] )
            total_layovers += len(self.possible_routes[y][1])
            total_time_diff += abs(int(self.arrival_time) - int(flight_list[int(self.possible_routes[y][1][-1])].arrival_time))

          for intermediate_flight in self.possible_routes[y][1] :
            flight_list[int(intermediate_flight)].empty_space -= route_flow

          y += 1
          z += route_flow


        alloted_passengers += int(z)


        for i in range (z,passenger_count):
          reallocated_passengers.append([self.passengers[i] , 0])






# Iterate through flight dataframe and update data of each flight in flight_list
for index, row in flights.iterrows():
    # Create a Flight object for each row in the flights DataFrame
    temp = Flight(row['FID'], row['DEP'], row['ARR'], row['DEP_TIME'], row['ARR_TIME'], row['CAPACITY'])
    flight_list.append(temp)

# Add passengers to their respective flights
for index, row in passengers.iterrows():
    flight_list[int(row['FID'])].add_passenger(row['PID'])

# If a flight has been canceled, we should not insert it in the graph
for index, row in canceled_flights.iterrows():
    flight_list[int(row['Canceled'])].cancel_flight()

# Creating a directed graph using airports as nodes
G = nx.MultiDiGraph()

for f in flight_list:
    G.add_node(f.departure_airport)
    G.add_node(f.arrival_airport)

# Create nodes with attribute id (flight id which connects the two nodes)
for f in flight_list:
    if not f.canceled:
        G.add_edge(f.departure_airport, f.arrival_airport, id=f.flight_id)


min_layover_time = int(5400)


# Do BFS for the given graph for each of the flights which have been canceled
for index, row in canceled_flights.iterrows():

    i = int(row['Canceled'])
    #calculation of stats
    affected_passengers += len(flight_list[i].passengers)
    ############################


    #Allotment of Flights

    src = flight_list[i].departure_airport  # source node
    tgt = flight_list[i].arrival_airport  # target node

    # Do BFS with this source
    path = [src]
    route = []
    valid_routes = []

    q = [[path, route]]
    # q is the queue which will store all the paths that we have created till now using BFS
    # We will update the valid_routes list if we get an alternate route for flight between departure and arrival airports which satisfies the constraints of <=3 layovers
    #BFS loop
    while q:
        rt_var = q.pop(0)

        path = rt_var[0]  #current path
        route = rt_var[1] #flight ids corresponding to the current path

        n = len(path)  #Tracks current path length
        last = path[-1] #last element of the current path

        if n == 1:
            for nb in G.neighbors(last):
                edge_data = G.get_edge_data(last, nb)

                if edge_data:
                    for key, edge in edge_data.items():
                        new_path = path.copy()
                        new_route = route.copy()
                        new_path.append(nb)
                        new_route.append(edge['id'])
                        if nb == tgt:
                            valid_routes.append([new_path, new_route])
                        else:
                            q.append([new_path, new_route])

        elif n == 2:
            f1 = int(route[0])
            for nb in G.neighbors(last):
                if nb == src:
                    continue
                edge_data = G.get_edge_data(last, nb)

                if edge_data:
                    for key, edge in edge_data.items():
                        f2 = int(edge['id'])

                        if min_layover_time <= int(flight_list[f2].departure_time)-int(flight_list[f1].arrival_time):
                            new_path = path.copy()
                            new_route = route.copy()
                            new_path.append(nb)
                            new_route.append(edge['id'])

                            if nb == tgt:
                                valid_routes.append([new_path, new_route])
                            else:
                                q.append([new_path, new_route])

        elif n == 3:
            f1 = int(route[1])

            for nb in G.neighbors(last):
                if nb == tgt:
                    edge_data = G.get_edge_data(last, tgt)

                    if edge_data:
                        for key, edge in edge_data.items():
                            f2 = int(edge['id'])

                            if min_layover_time <= int(flight_list[f2].departure_time)-int(flight_list[f1].arrival_time):
                                new_path = path.copy()
                                new_route = route.copy()
                                new_path.append(nb)
                                new_route.append(edge['id'])
                                valid_routes.append([new_path, new_route])

    flight_list[i].possible_routes = valid_routes
    flight_list[i].allocate_flights()


df = pd.DataFrame(reallocated_passengers , columns = ['PID' , 'Number of flights(layovers)' , 'Flights'] )
df.to_csv('allot.csv', index=False)
end = time.time()
execution_time = (end-start)*(10**3)

with open('stats.csv' , mode = 'w' , newline = '') as file :
  writer = csv.writer(file)
  writer.writerow(['Affected Passengers' , 'Relocated Passengers' , 'Average Layovers' , 'Average Time Differnce' , 'Execution Time (ms)'])
  writer.writerow([affected_passengers , alloted_passengers , total_layovers/alloted_passengers , total_time_diff/alloted_passengers, execution_time])

